In [ ]:
# import the necessary packages
import numpy as np
import IPython.display as disp

!pip install -U geemap
import geemap
!pip install earthengine-api

import matplotlib.pyplot as plt


In [ ]:
# authentication to gee
import ee

ee.Authenticate()
ee.Initialize(project="ee-bbensi")

In [ ]:
# east Emilia-Romagna
geometry = ee.Geometry.Point([12.180427, 44.696379])
roi = geometry.buffer(45000)

In [ ]:
# flood zone
geometry = ee.Geometry.Point([11.880427, 44.496379])
roi = geometry.buffer(20000)

In [ ]:
# Load Sentinel-2 image collection and filter by date and cloud cover.

# post-flood
input_post = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(roi) \
                .filterDate('2023-05-15', '2023-05-26') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .median()  # Create a median composite

# pre-flood
input_pre = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(roi) \
                .filterDate('2023-02-01', '2023-04-30') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .median()  # Create a median composite


input pre-flood in RGB

In [ ]:
# visualization pre-flood in RGB
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.5}

Map.addLayer(input_pre.clip(roi), vis_param, "overview pre-flood in RGB")
Map.centerObject(roi, 10)

Map

input pre-flood using B11, B8, B3

In [ ]:
# visualization pre-flood B11 B8 B3
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B11', 'B8', 'B3'],
             'gamma': 1.5}

Map.addLayer(input_pre.clip(roi), vis_param, "overview pre-flood B11 B8 B3")
Map.centerObject(roi, 10)

Map

input post-flood in RGB

In [ ]:
# visualization post-flood in RGB
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.5}

Map.addLayer(input_post.clip(roi), vis_param, "overview post-flood in RGB")
Map.centerObject(roi, 10)

Map

input post-flood using B11, B8, B3

In [ ]:
# visualization post-flood B11 B8 B3
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B11', 'B8', 'B3'],
             'gamma': 1.5}

Map.addLayer(input_post.clip(roi), vis_param, "overview post-flood B11 B8 B3")
Map.centerObject(roi, 10)

Map

# NDWI

NDWI pre-flood

In [ ]:
# Select the bands (B3, B8).
input2 = input_pre.select(['B3', 'B8'])

# Compute NDWI
ndwi_pre = input2.normalizedDifference(['B3', 'B8']).rename('NDWI_pre')

# Display the NDWI band (NDWI ranges from -1 to 1)
ndwiVis = {
  'min': -1.0,
  'max': 1.0,
  'palette': ['green', 'white', 'blue']
}

# Add NDWI to the map
m = geemap.Map()
m.addLayer(ndwi_pre.clip(roi), ndwiVis, 'NDWI_pre')
m.centerObject(roi, 10)
m

NDWI post-flood

In [ ]:
# Select the bands (B3, B8).
input2 = input_post.select(['B3', 'B8'])

# Compute NDWI
ndwi_post = input2.normalizedDifference(['B3', 'B8']).rename('NDWI_post')

# Display the NDWI band
ndwiVis = {
  'min': -1.0,
  'max': 1.0,
  'palette': ['green', 'white', 'blue']
}

# Add NDWI to the map
m = geemap.Map()
m.addLayer(ndwi_post.clip(roi), ndwiVis, 'NDWI_post')
m.centerObject(roi, 10)
m

# Change detection analysis

histogram pre-flood of NDWI

In [ ]:
# histogram pre-flood
# Define the region to get the histogram data
hist_region = roi

# ee.Reducer.fixedHistogram  to get the histogram
histogram = ndwi_pre.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(-1, 1, 50),  # Set range and number of bins
    geometry=hist_region,
    scale=10,
    bestEffort=True
)

# Get the histogram data
hist_data = ee.List(histogram.get('NDWI_pre')).getInfo()

# Extract bins and counts
bins = [d[0] for d in hist_data]
counts = [d[1] for d in hist_data]

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(bins, counts, width=(bins[1]-bins[0]), align='center', edgecolor='k')
plt.xlabel('NDWI pre-flood')
plt.ylabel('Count')
plt.title('Histogram of NDWI pre-flood')
plt.show()

histogram post-flood of NDWI

In [ ]:
# histogram post-flood
# Define the region to get the histogram data
hist_region = roi

# ee.Reducer.fixedHistogram to get the histogram
histogram = ndwi_post.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(-1, 1, 50),  # Set range and number of bins
    geometry=hist_region,
    scale=10,
    bestEffort=True
)

# Get the histogram data
hist_data = ee.List(histogram.get('NDWI_post')).getInfo()

# Extract bins and counts
bins = [d[0] for d in hist_data]
counts = [d[1] for d in hist_data]

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(bins, counts, width=(bins[1]-bins[0]), align='center', edgecolor='k')
plt.xlabel('NDWI post-flood')
plt.ylabel('Count')
plt.title('Histogram of NDWI post-flood')
plt.show()

ratio of NDWi

In [ ]:
#ee.Image.divide to get the ratio
ratio = ndwi_post.divide(ndwi_pre).rename('Ratio')
m = geemap.Map()
m.addLayer(ratio.clip(roi))
m.centerObject(roi, 10)
m

In [ ]:
# Compute the histogram of the ratio
histogram = ratio.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(-10, 10, 100),
    geometry=roi,
    scale=10,
    bestEffort=True
)

# Get the histogram data
hist_dict = histogram.get('Ratio').getInfo()

# Extract bins and counts
bins = [d[0] for d in hist_dict]
counts = [d[1] for d in hist_dict]

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(bins, counts, width=(bins[1]-bins[0]), align='center', edgecolor='k')
plt.xlabel('Ratio')
plt.ylabel('Frequency')
plt.title('Histogram of Ratio')
plt.show()

In [ ]:
# Calculate mean and standard deviation of the ratio
mean = ratio.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry= roi,
  scale= 10,
  bestEffort= True
)

std_dev = ratio.reduceRegion(
  reducer= ee.Reducer.stdDev(),
  geometry= roi,
  scale= 10,
  bestEffort= True
)

# Get the mean and standard deviation values
mean = mean.get('Ratio').getInfo()
std_dev = std_dev.get('Ratio').getInfo()

print(f'Mean: {mean:.4f}')
print(f'Standard Deviation: {std_dev:.4f}')

In [ ]:
# compute the minimum and maximum ratio values
min = ratio.reduceRegion(
  reducer= ee.Reducer.min(),
  geometry= roi,
  scale= 10,
  bestEffort= True
)

max = ratio.reduceRegion(
  reducer= ee.Reducer.max(),
  geometry= roi,
  scale= 10,
  bestEffort= True
)

# Get the min and max values
min_value = min.get('Ratio').getInfo()
max_value = max.get('Ratio').getInfo()

print(f'Min: {min_value:.4f}')
print(f'Max: {max_value:.4f}')

In [ ]:
# Concatenate the images to work with both NDWI bands
combined_ndwi = ee.Image.cat(ndwi_pre.rename('NDWI_pre'), ndwi_post.rename('NDWI_post'))

# Define conditions for each category
category_image = combined_ndwi.expression(
  """
  (b('NDWI_post') > 0) && (b('NDWI_pre') > 0) && (b('NDWI_post') > b('NDWI_pre')) ? 1 :
  (b('NDWI_post') > 0) && (b('NDWI_pre') > 0) && (b('NDWI_post') < b('NDWI_pre')) ? 2 :
  (b('NDWI_post') > 0) && (b('NDWI_pre') < 0) ? 3 :
  (b('NDWI_post') < 0) && (b('NDWI_pre') > 0) ? 4 :
  (b('NDWI_post') < 0) && (b('NDWI_pre') < 0) && (b('NDWI_post') < b('NDWI_pre')) ? 5 :
  (b('NDWI_post') < 0) && (b('NDWI_pre') < 0) && (b('NDWI_post') > b('NDWI_pre')) ? 6 :
  0
  """,
  {
    'NDWI_pre': combined_ndwi.select('NDWI_pre'),
    'NDWI_post': combined_ndwi.select('NDWI_post')
  }).rename('Change_Category')


In [ ]:
# Visualization palette for the categories
palette = ['blue', 'green', 'cyan', 'magenta', 'red', 'yellow', 'black']

# Create a map and add the categorized image layer
m = geemap.Map()
m.addLayer(category_image.clip(roi), {'min': 1, 'max': 7, 'palette': palette}, 'Change Categories')
m.centerObject(roi, 10)
m


In [ ]:
# Function to compute the area in square meters for each category
def compute_area_for_category(category_id, category_image):
    mask = category_image.eq(category_id)
    category_area = ee.Image.pixelArea().updateMask(mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('area')
    return ee.Number(category_area).getInfo()

# Compute total area for each category
total_area_category_1 = compute_area_for_category(1, category_image)
total_area_category_2 = compute_area_for_category(2, category_image)
total_area_category_3 = compute_area_for_category(3, category_image)
total_area_category_4 = compute_area_for_category(4, category_image)
total_area_category_5 = compute_area_for_category(5, category_image)
total_area_category_6 = compute_area_for_category(6, category_image)
total_area_category_0 = compute_area_for_category(0, category_image)


In [ ]:
# Print the area for each category in square meters
print(f'Total area for Category 1: {total_area_category_1} square meters')
print(f'Total area for Category 2: {total_area_category_2} square meters')
print(f'Total area for Category 3: {total_area_category_3} square meters')
print(f'Total area for Category 4: {total_area_category_4} square meters')
print(f'Total area for Category 5: {total_area_category_5} square meters')
print(f'Total area for Category 6: {total_area_category_6} square meters')
print(f'Total area for Category 0: {total_area_category_0} square meters')



In [ ]:
# Print the area for each category in square kilometers
print(f'Total area for Category 1: {total_area_category_1 / 1e6} square kilometers')
print(f'Total area for Category 2: {total_area_category_2 / 1e6} square kilometers')
print(f'Total area for Category 3: {total_area_category_3 / 1e6} square kilometers')
print(f'Total area for Category 4: {total_area_category_4 / 1e6} square kilometers')
print(f'Total area for Category 5: {total_area_category_5 / 1e6} square kilometers')
print(f'Total area for Category 6: {total_area_category_6 / 1e6} square kilometers')
print(f'Total area for Category 0: {total_area_category_0 / 1e6} square kilometers')

difference of NDWI index

In [ ]:
# Image.subtract(image2) to get the difference
difference = ndwi_pre.subtract(ndwi_post).rename('Difference')
m = geemap.Map()
m.addLayer(difference.clip(roi))
m.centerObject(roi, 10)
m

change vector analysis

In [ ]:
# Choose bands that are sensitive to the changes due to the flood
bands = ['B8', 'B11']

# Calculate change vectors
change_vectors = input_post.select(bands).subtract(input_pre.select(bands))

# Calculate magnitude of change
magnitude = change_vectors.pow(2).reduce(ee.Reducer.sum()).sqrt().clip(roi)

# Calculate direction of change
direction = change_vectors.select('B8').atan2(change_vectors.select('B11')).clip(roi)


# Visualize results
magnitude_vis = {
    'min': 0,
    'max': 3000,  # based on histogram result
    'palette': ['blue', 'green', 'yellow']
}

direction_vis = {
    'min': -1.57,  # -90 degrees in radians
    'max': 1.57,  # 90 degrees in radians
    'palette': ['orange', 'white', 'purple']
}

# Add layers to the map
map = geemap.Map()
map.addLayer(magnitude, magnitude_vis ,'Magnitude of Change')
map.addLayer(direction, direction_vis ,'Direction of Change')
map.centerObject(roi, 10)
map

In [ ]:
# Compute the histogram of the magnitude
histogram = magnitude.reduceRegion(
    reducer=ee.Reducer.histogram(),
    geometry=roi,
    scale=20,
    bestEffort=True
)

# Get the histogram data
hist_dict = histogram.get('sum').getInfo()

# Extract the bucketMeans (x-axis values) and histogram (y-axis values)
bucketMeans = hist_dict['bucketMeans']
histogram_counts = hist_dict['histogram']

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.bar(bucketMeans, histogram_counts, width=hist_dict['bucketWidth'],  align='center', edgecolor='k')
plt.xlabel('Magnitude of Change')
plt.ylabel('Frequency')
plt.title('Histogram of Magnitude')
plt.show()
plt.savefig('histogram.png')

# Unsupervised

unsupervised learning on the NDWI difference

LVQ

In [ ]:
m = geemap.Map()
m.addLayer(ee.Image().paint(roi, 0, 2), {}, 'region')

# Make the training dataset
diff_clipped = difference.clip(roi)
training = diff_clipped.sample(region=roi, scale=10, numPixels=8000)

# Instantiate the clusterer and train it
clusterer = ee.Clusterer.wekaLVQ(2).train(training)

# Cluster the input using the trained clusterer
result_LVQ = diff_clipped.cluster(clusterer)

# Define a palette
palette = ['black', 'purple']
# Visualize clusters using the defined palette
m.add_basemap('HYBRID')
m.addLayer(result_LVQ, {'min': 0, 'max': len(palette) - 1, 'palette': palette, 'opacity': 0.5}, 'clusters')


m.centerObject(roi, 10)

# Display the map.
m


KMEANS

In [ ]:
m = geemap.Map()
m.addLayer(ee.Image().paint(roi, 0, 2), {}, 'region')

# Make the training dataset
diff_clipped_K = difference.clip(roi)
training = diff_clipped_K.sample(region=roi, scale=10, numPixels=8000)

# Instantiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(nClusters = 2, init = 1).train(training) # init = 1 stands for k-means++ cluster centers initialization

# Cluster the input using the trained clusterer
result_KMEANS = diff_clipped_K.cluster(clusterer)

# Define a palette
palette = ['black', 'blue']
# Visualize clusters using the defined palette
m.add_basemap('HYBRID')
m.addLayer(result_KMEANS, {'min': 0, 'max': len(palette) - 1, 'palette': palette, 'opacity': 0.5}, 'clusters')


m.centerObject(roi, 10)

# Display the map.
m

difference between KMEANS and LVQ results

In [ ]:
# Create a difference image between Kmeans and LVQ results
difference_image = result_KMEANS.subtract(result_LVQ)

# Define a palette
palette_diff = ['red', 'black', 'red']

# Add the difference image to the map
m = geemap.Map()
m.add_basemap('HYBRID')
m.add_layer(difference_image, {'min': -1, 'max': 1, 'palette': palette_diff, 'opacity': 0.5}, 'Difference (KMeans - LVQ)')

m.centerObject(roi, 10)
m

compute the area of the flooded cluster

 LVQ

In [ ]:
# Define the cluster number representing the flooded area
flooded_cluster = 1

# Create a binary mask of the flooded areas
flooded_mask = result_LVQ.eq(flooded_cluster)

# Calculate the pixel area in square meters
pixel_area = ee.Image.pixelArea()

# Calculate the area of flooded regions by multiplying the binary mask by the pixel area
flooded_area = flooded_mask.multiply(pixel_area)

# Sum the flooded areas within the region of interest
flooded_area_stats = flooded_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=10,
    maxPixels=1e9
)


# Print the keys
print(flooded_area_stats.keys().getInfo())

# Extract the value from the output dictionary
flooded_area_m2 = flooded_area_stats.get('cluster').getInfo()
print(f'Flooded area (in square meters): {flooded_area_m2}')

# Convert flooded area from square meters to square kilometers
flooded_area_km2 = flooded_area_m2 / 1e6
print(f"Flooded area: {flooded_area_km2:.2f} square kilometers")


per KMEANS

In [ ]:
# Define the cluster number representing the flooded area
flooded_cluster = 1

# Create a binary mask of the flooded areas
flooded_mask = result_KMEANS.eq(flooded_cluster)

# Calculate the pixel area in square meters
pixel_area = ee.Image.pixelArea()

# Calculate the area of flooded regions by multiplying the binary mask by the pixel area
flooded_area = flooded_mask.multiply(pixel_area)

# Sum the flooded areas within the region of interest
flooded_area_stats = flooded_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=10,
    maxPixels=1e9
)

# Print the keys
print(flooded_area_stats.keys().getInfo())

# Extract the value from the output dictionary
flooded_area_m2 = flooded_area_stats.get('cluster').getInfo()
print(f'Flooded area (in square meters): {flooded_area_m2}')

# Convert flooded area from square meters to square kilometers
flooded_area_km2 = flooded_area_m2 / 1e6
print(f"Flooded area: {flooded_area_km2:.2f} square kilometers")

